# START

In [1]:
import numpy as np
import pandas as pd

In [2]:
data= pd.read_csv("A_Z Handwritten Data.csv") #Reading the file and making a pandas dataframe

In [3]:
data.shape #printing the number of rows and columns in our Dataframe

(372450, 785)

SPLITTING THE DATA INTO TRAIN AND TEST SET

In [4]:
from sklearn.model_selection import train_test_split #importing the train_test_split function to split the data

In [5]:
letterVal = data.iloc[:,[0]].copy() #Storing the Data of Letters in letterVal. This is the output which is expected
data=data.drop(data.columns[0],axis=1) #Dropping the expected output columns from data. Now data stores the features (Pixels)

In [6]:
data.shape 

(372450, 784)

In [7]:
letterVal.shape 

(372450, 1)

# Training the Model:

# Splitting the data set

In [8]:
X_train, X_test, Y_train, Y_test =train_test_split(data,letterVal,test_size=0.1)


In [9]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(335205, 784)
(37245, 784)
(335205, 1)
(37245, 1)


In [10]:
forPrior = Y_train.copy() #Creating a separate dataframe to work on, so that the original value is not modified in the process

In [11]:
numCount = np.ones(26) #creating an array of size 26, which will store the number of occurrence of each letter

In [12]:
n=0
while(n<26):
    numCount[n]=forPrior.value_counts()[n]
    n=n+1
print(numCount)

[12562.  7809. 21121.  9091. 10319.  1044.  5231.  6485.  1007.  7596.
  5001. 10434. 11067. 17055. 52050. 17412.  5244. 10378. 43664. 20223.
 26081.  3772.  9699.  5602.  9773.  5485.]


In [13]:
len2=forPrior.shape[0]
len2

335205

# Finding Prior

In [14]:
prior=np.ones(26)
n=0
while(n<26):
    prior[n]=numCount[n]/len2
    n=n+1
print(prior)

[0.03747557 0.02329619 0.0630092  0.02712072 0.03078415 0.00311451
 0.01560538 0.01934637 0.00300413 0.02266076 0.01491923 0.03112722
 0.03301562 0.05087931 0.15527811 0.05194433 0.01564416 0.03096016
 0.13026059 0.06033025 0.07780612 0.01125282 0.02893453 0.01671216
 0.02915529 0.01636312]


In [15]:
arr = X_train.to_numpy(None,True) #True means that we are making a copy of X_train, so that any updation on arr doesn't change 
#values in X_train

# Scaling down the multinomial into binomial

The count0 array is of size 26x784, where 26 is the number of rows, each row representing a letter, and 784 columns represents the 784 pixels of an image. This array stores the number of times 0 occurs for a pixel number of a letter.
For instance count0[0][0] would store the number of times the 0th pixel of letter A was 0, out of the total number of time letter A occured in the data set :

In [16]:
count0=np.zeros([26,784])

Scaling down and Counting the number of zeros: 

In [17]:
rows=len(arr)
columns=len(arr[0])
r=0
while(r<rows):
    c=0
    while(c<columns):
        if(arr[r][c]<128):
            arr[r][c]=0
        else:
            arr[r][c]=1
            
        if(arr[r][c]==0): 
            x=Y_train.iloc[r,0] #since this cell is zero, we find which number this row corresponds to
            count0[x][c]=count0[x][c]+1 #now X stores the letter for which we found a 0, so we increase the count of the 
            #corresponding cell in count0 array.
        c=c+1
    r=r+1


In [18]:
#n=0
#while(n<rows):
#    c=0
#    while(c<columns):
        
        
#        c=c+1
 #   n=n+1
    

In [19]:
count1=np.zeros([26,784]) #like count0, this array will store the number of occurences of 1 for a pixel of a particular image

# Smoothing

In [20]:
c=0
r=0
flag=0
while(r<26):
    c=0
    while(c<784):
        if(count0[r][c]==numCount[r]): #if number of occurences of 0 for a pixel is same as the number of times that letter appeared
            count0[r][c]=numCount[r]-1 #we decrease the count by 1
        c=c+1
    r=r+1

Finding the number of times 1 appeared for a pixel: 

In [21]:
print(count0[0][0])

12561.0


In [22]:
n=0
c=0
while(n<26):
    c=0
    while(c<784):
        count1[n][c]=numCount[n]-count0[n][c] #subtracting number of times 0 appeared from total number of times the letter appeared
        c=c+1
    n=n+1

# Finding conditional probabilities of a pixel being 1 or 0 given a letter

In [23]:
print(count1[0][0])

1.0


In [24]:
conditional0=np.zeros([26,784])  #a 2D array which will store the conditional probability of a cell being 0 given that a letter appeared

In [25]:

n=0
while(n<26):
    c=0
    while(c<784):
        conditional0[n][c]=count0[n][c]/numCount[n]
        c=c+1
    n=n+1


In [26]:
conditional1 = np.zeros([26,784]) #a 2D array which will store the conditional probability of a cell being 1 given that a letter appeared

Finding the conditional probability of a pixel being 1 given that letter 'n' has occured:

In [27]:
n=0
while(n<26):
    c=0
    while(c<784):
        conditional1[n][c]=count1[n][c]/numCount[n]
        c=c+1
    n=n+1

In [28]:
logconditional1=np.log(conditional1) #creating an array from conditional1, storing the log values of corresponding values

In [29]:
logprior=np.log(prior) #creating an array from prior, storing the log values of correponding values

In [30]:
logconditional0=np.log(conditional0) #creating an array from conditional, storing the log values

# Testing our Model

NOTE: we don't need to find the total probability because basically (a/b > c/b) is same as (a>b)

Defining a function for finding the probability:

In [37]:
def findprob(a,k): # 'a' is the data point of size 784, containing the features using which we have to find the probability
    #of 'a' representing letter 'k'
    n=0
    x=0
    while(n<784):
        if(a[n]==0): #if the current feature is 0
            x=x+logconditional0[k][n] #since we are multiplying the log values, and in log, log(a*b)=log(a)+log(b)
#in the previous line, logconditional contains the conditional probability of a cell being 0
        elif(a[n]==1): #if the current feature is 1
            x=x+logconditional1[k][n]
#in the previous line, logconditonal1 contains the condtionl probability of a cell being 1           
        n=n+1
    
    x=x+logprior[k] #adding the prior probability of k
    
    return x

Defining a function forall() which will take a datapoint and find the probabilities of the data point representing letters A to Z:

In [32]:
def forall(X): #X is the data point (features) of size 784 pixels
    prob1=np.zeros(26) #creating an array of size 26, which will store the probabilities of the data point X representing the
    #letters A to Z, stored in the corresponding indexes 0 to 25
    n=0
    while(n<26): 
        prob1[n]=findprob(X,n) #calling the findprob function and passing the data point and the letter represented by n
        n=n+1
        
    return pd.Series(prob1).idxmax() #will return the index of the maximum value stored in the array prob1

Creating the copy of the Test Data, so that we don't end up modifying the data of our test data set

In [33]:
testarr=X_test.to_numpy(None,copy=True) #testarr is a 2-D array of size same as our X_test

Now, before running the algorithm on our testarr, we have to scale down the features into binomial, i.e. 0 for feature being less than 128 and 1 otherwise

In [34]:
numRow=testarr.shape[0]
numCol=testarr.shape[1]
n=0
c=0
while(n<numRow):
    c=0
    while(c<numCol):
        if(testarr[n][c]<128):
            testarr[n][c]=0
        else:
            testarr[n][c]=1
        c=c+1
    n=n+1

In [35]:
testarr.shape

(37245, 784)

# Passing the Test data set to our algorithm

op is an array of same size as the size of test data set, it will store the output values of the corresponding feature of the test data, and later on we will compare the values of op with the Y_test data set, which stores the actual output of the corrensponding features.

In [38]:

n=0
Predicted_OP=np.zeros(37245)
vec=np.zeros(784) #vec will store the 784 features of the 'n'th data point and we will pass this to our function forall
while(n<37245):
    vec=testarr[n].copy() #copying the features of our n'th data point in vec
    Predicted_OP[n]=forall(vec)
    n=n+1

# Performance analysis:

In [39]:
confusion_matrix=np.zeros([26,26]) #creating a confusion_matrix
count=0
n=0
while(n<Y_test.shape[0]):
    x=int(Y_test.iloc[n,0])
    y=int(Predicted_OP[n])
    confusion_matrix[x,y]+=1
    if(Predicted_OP[n]==Y_test.iloc[n,0]):
        count=count+1 #if our predicted output and actual output is same, increase the count value by 1
    n=n+1


In [40]:
confusion_matrix=confusion_matrix.astype(int) #converting the confusion matrix into values of int, 
#so that we don't get output in Scientific notations

# Printing the Confusion Matrix

In [41]:
print(confusion_matrix)

[[ 929   21    0    0    3    0   12   60    0    3    8    0   59   44
     1   17   20   37    0    0    2    0   15   42   34    0]
 [  30  610    5   23   20    0    9   13    1   24    1    2    9    9
    25   10    5    7   16    0    0    0    2    4   14   20]
 [   1   28 1601    3   86    6   33    4    0   18   30   97   28   20
   131   52   17    6    0   13   32    0   66    0   11    5]
 [  12   45    4  725    1    0    0    1    0   55    0    9    6    8
    93   11   11    0   11    1   20    0   12    0    3   15]
 [  17   45   38    0  659   10   29    0    3    3   80   60   12   20
     5   37    3   51   11   12    4    0    5    2    9    6]
 [   0    0    0    0    6  102    0    0    0    1    0    0    0    0
     0    8    0    0    0    1    0    0    0    0    1    0]
 [   2   11   10    2   12    0  382    1    0    3    1    5    4    3
     7    2   42    3   20    1    9    1    7    0    2    1]
 [  44   11    0    0    0    0    4  404    0    2    

# Finding the True Positive, True Negative, False Positive and False Negative For each Letter

In [43]:
n=0
TpTnFpFn=np.zeros([26,4])        
while(n<26):
    Actual_OP=0
    while(Actual_OP<26):
        Predicted_Val=0
        while(Predicted_Val<26):
            if(n==Actual_OP):
                if(Actual_OP==Predicted_Val):
                    TpTnFpFn[n][0]=confusion_matrix[Actual_OP][Predicted_Val]
                else:
                    TpTnFpFn[n][3]+=confusion_matrix[Actual_OP][Predicted_Val]
            elif(Actual_OP!=n):
                if(Predicted_Val==n):
                    TpTnFpFn[n][2]+=confusion_matrix[Actual_OP][Predicted_Val]
                else:
                    TpTnFpFn[n][1]+=confusion_matrix[Actual_OP][Predicted_Val]
            Predicted_Val+=1
        Actual_OP+=1
    n+=1
                
                
                    
        

In [44]:
TpTnFpFn=TpTnFpFn.astype(int)

Printing the TP, TN, FP, FN for each letter: 

In [45]:
print(TpTnFpFn)

[[  929 35400   538   378]
 [  610 35683   703   249]
 [ 1601 34641   316   687]
 [  725 35590   612   318]
 [  659 35861   263   462]
 [  102 36963   163    17]
 [  382 36371   343   149]
 [  404 36015   497   329]
 [   95 36701   431    18]
 [  526 35539   809   371]
 [  409 36166   477   193]
 [  827 35823   270   325]
 [ 1107 35515   461   162]
 [ 1251 34510   780   704]
 [ 4541 30996   474  1234]
 [ 1408 34958   358   521]
 [  390 36379   298   178]
 [  640 35720   337   548]
 [ 3346 32370   120  1409]
 [ 1594 34619   354   678]
 [ 1867 33962   356  1060]
 [  346 36697   138    64]
 [  843 35445   715   242]
 [  480 36247   328   190]
 [  795 35451   708   291]
 [  360 36495   159   231]]


# Classwise/Letterwise Accuracy, Sensitivity and Precision:

In [46]:
accuracy=np.zeros(26)
sensitivity=np.zeros(26)
precision=np.zeros(26)
n=0
while(n<26):
    accuracy[n]=(TpTnFpFn[n][0]+TpTnFpFn[n][1])/(TpTnFpFn[n][0]+TpTnFpFn[n][1]+TpTnFpFn[n][2]+TpTnFpFn[n][3])
    sensitivity[n]=(TpTnFpFn[n][0])/(TpTnFpFn[n][0]+TpTnFpFn[n][3])
    precision[n]=(TpTnFpFn[n][0])/(TpTnFpFn[n][0]+TpTnFpFn[n][2])
    n=n+1

In [47]:
print(accuracy)

[0.97540609 0.97443952 0.97307021 0.97503021 0.9805343  0.99516714
 0.98679017 0.97782253 0.98794469 0.9683179  0.98201101 0.9840247
 0.98327292 0.96015573 0.9541415  0.97639952 0.98721976 0.97623842
 0.95894751 0.97229158 0.96198147 0.99457645 0.97430528 0.98609209
 0.97317761 0.9895288 ]


In [48]:
print(sensitivity)

[0.71078806 0.71012806 0.69973776 0.69511026 0.58786798 0.85714286
 0.71939736 0.55115962 0.84070796 0.58639911 0.67940199 0.71788194
 0.87234043 0.6398977  0.78632035 0.72991187 0.68661972 0.53872054
 0.70368034 0.70158451 0.63785446 0.84390244 0.77695853 0.71641791
 0.7320442  0.60913706]


In [49]:
print(precision)

[0.63326517 0.46458492 0.8351591  0.54225879 0.71475054 0.38490566
 0.52689655 0.44839068 0.18060837 0.39400749 0.46162528 0.7538742
 0.7059949  0.61595273 0.90548355 0.79728199 0.56686047 0.65506653
 0.96537796 0.81827515 0.83985605 0.71487603 0.54107831 0.59405941
 0.52894212 0.69364162]


In [50]:
print(count) #will print the total number of correctly predicted output

26237


In [52]:
print("Overall accuracy of model: ",100*count/Y_test.shape[0]) #will print the percentage of correctly predicted output, i.e. the accuracy of our model

Overall accuracy of model:  70.44435494697275
